In [3]:
import pandas as pd
from itertools import combinations
from pymatgen.core import Composition
from itertools import combinations

def conditional_dfs(df, reference_formula, out_cols):

    ref_dict = Composition(reference_formula).get_el_amt_dict()
    mask = df["formula_pretty"].apply(lambda f: Composition(f).get_el_amt_dict() != ref_dict)
    df_filtered = df[mask].reset_index(drop=True)

    elements = sorted(ref_dict.keys())
   
    df_filtered["comps"] = df_filtered["formula_pretty"].apply(lambda f: set(Composition(f).get_el_amt_dict().keys()))

    dfs = {}

    #has_A
    for el in elements:
        dfs[f"has_{el}"] = df_filtered[df_filtered["comps"].apply(lambda S: el in S)][out_cols] 

    #has A_or_B
    for el1, el2 in combinations(elements, 2):
        dfs[f"{el1}_or_{el2}"] = df_filtered[
        df_filtered["comps"].apply(lambda S: (el1 in S) or (el2 in S))][out_cols]
        
    #gather all index from has_A and has A_or_B
    used_idx = pd.Index([])
    for d in dfs.values():
        used_idx = used_idx.union(d.index)
    #none
    dfs["none"] = df_filtered.drop(index=used_idx)[out_cols]
    
    return dfs, df_filtered[out_cols]


In [4]:
reference_formula = "AcClO"  
df = pd.read_csv("datasets/129_ABC_mp-30273.csv")

dfs, df_no_ref = conditional_dfs(df,reference_formula=reference_formula,
    out_cols=["formula_pretty", "band_gap"])

for condition, data_length in dfs.items():
    print(condition, len(data_length))

print(dfs.keys())

#get a specific dataframe
df_none = dfs["none"]
print(df_none.head())




has_Ac 1
has_Cl 40
has_O 68
Ac_or_Cl 41
Ac_or_O 68
Cl_or_O 90
none 260
dict_keys(['has_Ac', 'has_Cl', 'has_O', 'Ac_or_Cl', 'Ac_or_O', 'Cl_or_O', 'none'])
   formula_pretty  band_gap
10         RbCaAs    1.2985
14           KMgP    1.6589
17          TaSeN    0.0000
21          NbSeN    0.0000
23         CsMnBi    0.0000
